# AI Agent with Tools and Threads Demo

This notebook demonstrates how to create AI agents with custom tools and manage conversation threads, similar to the Azure OpenAI Assistants API pattern.

## Features:
- **Custom Tools**: Weather and Calculator tools
- **Thread Management**: Separate conversation contexts
- **Tool Detection**: Automatic tool usage based on user input
- **Tracing Integration**: Full observability with OpenTelemetry

In [ ]:
# Import required libraries
import asyncio
import os
import time
from typing import Any, Dict, List
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Import Azure AI agent framework
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# Import Azure AI agent models for tools
from azure.ai.agents.models import FunctionTool, ToolSet

# Import agent framework for workflows and tracing
try:
    from agent_framework import Executor, WorkflowBuilder, WorkflowContext, handler
    from agent_framework.observability import setup_observability
except ImportError:
    print("⚠️  Agent framework not available - continuing without it")

print("✅ All libraries imported successfully")

✅ All libraries imported successfully


In [2]:
# Setup tracing for AI Foundry
try:
    # Try to import and setup observability
    from agent_framework.observability import setup_observability
    setup_observability(
        otlp_endpoint="http://localhost:4317",
        enable_sensitive_data=True
    )
    print("✅ Tracing configured for AI Foundry")
except ImportError as e:
    print(f"⚠️  Agent framework not available: {e}")
    print("Continuing without agent framework tracing...")
except Exception as e:
    print(f"⚠️  Tracing setup failed: {e}")
    print("Continuing without tracing...")

# Initialize Azure AI Projects client
try:
    # Check which environment variable is available
    project_endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT") or os.getenv("PROJECT_ENDPOINT")
    connection_string = os.getenv("AIPROJECT_CONNECTION_STRING")
    
    if connection_string:
        # Use connection string method
        client = AIProjectClient.from_connection_string(
            conn_str=connection_string,
            credential=DefaultAzureCredential()
        )
        print("✅ Azure AI Projects client initialized using connection string")
    elif project_endpoint:
        # Use endpoint method
        client = AIProjectClient(
            endpoint=project_endpoint,
            credential=DefaultAzureCredential()
        )
        print("✅ Azure AI Projects client initialized using endpoint")
    else:
        print("❌ No Azure AI Project configuration found")
        print("Please set either:")
        print("  - AIPROJECT_CONNECTION_STRING, or")
        print("  - AZURE_AI_PROJECT_ENDPOINT (or PROJECT_ENDPOINT)")
        client = None
        
except Exception as e:
    print(f"⚠️  Azure AI Projects client setup failed: {e}")
    print("Please check your environment variables and Azure authentication")
    client = None

✅ Tracing configured for AI Foundry
✅ Azure AI Projects client initialized using endpoint


In [ ]:
# Define custom tools for the agent
def get_weather(location: str) -> str:
    """Get the weather for a given location"""
    weather_data = {
        "Amsterdam": "cloudy with a high of 15°C",
        "New York": "sunny with a high of 22°C", 
        "London": "rainy with a high of 12°C",
        "Tokyo": "partly cloudy with a high of 18°C",
        "Sydney": "sunny with a high of 25°C",
        "Paris": "overcast with a high of 14°C",
        "Berlin": "snowy with a high of 3°C"
    }
    weather = weather_data.get(location, f"partly cloudy with a high of 20°C")
    return f"The weather in {location} is {weather}."

def calculate(expression: str) -> str:
    """Perform mathematical calculations"""
    try:
        # Basic safety check
        allowed_chars = set('0123456789+-*/()., ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error calculating {expression}: {str(e)}"

# Create FunctionTool instances for Azure AI agents
weather_functions = FunctionTool(functions={get_weather})
calculator_functions = FunctionTool(functions={calculate})
combined_functions = FunctionTool(functions={get_weather, calculate})

# Create ToolSets for the agents
weather_toolset = ToolSet()
weather_toolset.add(weather_functions)

calculator_toolset = ToolSet()
calculator_toolset.add(calculator_functions)

combined_toolset = ToolSet()
combined_toolset.add(combined_functions)

print("✅ Tool functions and toolsets defined")
print("🧪 Testing tools:")
print(f"Weather: {get_weather('Amsterdam')}")
print(f"Calculator: {calculate('15 * 8 + 12')}")

✅ Tool functions defined
🧪 Testing tools:
Weather: The weather in Amsterdam is cloudy with a high of 15°C.
Calculator: The result of 15 * 8 + 12 is 132


In [ ]:
# Create AI Agents using Azure AI agent framework
async def create_agents():
    """Create weather and calculator agents"""
    if not client:
        print("❌ Cannot create agents - Azure AI Projects client not available")
        return None, None, None
    
    try:
        # Create Weather Agent
        print("🌤️  Creating Weather Agent...")
        weather_agent = await client.agents.create_agent(
            model=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="WeatherAgent", 
            instructions="You are a helpful weather assistant. Use the get_weather tool to provide weather information for any location the user asks about.",
            toolset=weather_toolset
        )
        print(f"✅ Weather Agent created: {weather_agent.id}")
        
        # Create Calculator Agent  
        print("🧮 Creating Calculator Agent...")
        calculator_agent = await client.agents.create_agent(
            model=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="CalculatorAgent",
            instructions="You are a helpful calculator assistant. Use the calculate tool to perform mathematical calculations for the user.",
            toolset=calculator_toolset
        )
        print(f"✅ Calculator Agent created: {calculator_agent.id}")
        
        # Create Main Agent that can use both tools
        print("🤖 Creating Main Agent with both tools...")
        main_agent = await client.agents.create_agent(
            model=os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o"),
            name="MainAssistant",
            instructions="""You are a helpful assistant with access to weather and calculator tools.
            
Available tools:
- get_weather: Get weather information for any location
- calculate: Perform mathematical calculations

When users ask about weather, use the get_weather tool.
When users ask for calculations, use the calculate tool.
Always provide helpful and clear responses.""",
            toolset=combined_toolset
        )
        print(f"✅ Main Agent created: {main_agent.id}")
        
        return weather_agent, calculator_agent, main_agent
        
    except Exception as e:
        print(f"❌ Error creating agents: {e}")
        print(f"Full error details: {type(e).__name__}: {str(e)}")
        return None, None, None

# Create the agents
weather_agent, calculator_agent, main_agent = await create_agents()

🌤️  Creating Weather Agent...
❌ Error creating agents: 'dict' object has no attribute 'file_search'


In [ ]:
# Create conversation threads and run agent conversations
async def run_agent_conversation(agent, message: str, thread_id: str = None):
    """Run a conversation with an AI agent"""
    if not client or not agent:
        print("❌ Cannot run conversation - client or agent not available")
        return None, None
        
    try:
        # Import tracing if available
        try:
            from opentelemetry import trace
            tracer = trace.get_tracer(__name__)
        except ImportError:
            tracer = None
        
        # Create thread if not provided
        if not thread_id:
            thread = await client.agents.threads.create()
            thread_id = thread.id
            print(f"📝 Created new thread: {thread_id}")
        
        # Add user message to thread
        await client.agents.messages.create(
            thread_id=thread_id,
            role="user", 
            content=message
        )
        
        # Run the agent on the thread and process it
        if tracer:
            with tracer.start_as_current_span("agent_run") as run_span:
                run_span.set_attribute("agent.id", agent.id)
                run_span.set_attribute("agent.name", agent.name)
                run_span.set_attribute("user_message", message)
                
                run = await client.agents.runs.create_and_process(
                    thread_id=thread_id, 
                    agent_id=agent.id
                )
                run_span.set_attribute("run.status", run.status)
        else:
            run = await client.agents.runs.create_and_process(
                thread_id=thread_id, 
                agent_id=agent.id
            )
        
        print(f"🔄 Run finished with status: {run.status}")
        
        # Check if the run failed
        if run.status == "failed":
            error_msg = f"Run failed: {run.last_error if hasattr(run, 'last_error') else 'Unknown error'}"
            print(f"❌ {error_msg}")
            return error_msg, thread_id
        
        # Get the response messages
        messages = await client.agents.messages.list(thread_id=thread_id)
        
        # Find the latest assistant message
        assistant_response = "No response received"
        for message in messages.data:
            if message.role == "assistant":
                # Get the text content from the message
                if hasattr(message, 'content') and message.content:
                    if hasattr(message.content[0], 'text'):
                        assistant_response = message.content[0].text.value
                    else:
                        assistant_response = str(message.content[0])
                break
        
        return assistant_response, thread_id
        
    except Exception as e:
        print(f"❌ Error in agent conversation: {e}")
        print(f"Full error details: {type(e).__name__}: {str(e)}")
        return f"Error: {str(e)}", None

print("✅ Agent conversation function ready")

In [ ]:
# Demo: AI Agents with Tools and Threads
async def demo_agents_with_tools():
    """Demonstrate AI agents with tools and threads like in your C# example"""
    print("🤖 AI Agents with Tools and Threads Demo")
    print("=" * 50)
    
    if not main_agent:
        print("❌ Agents not available - check Azure AI Projects configuration")
        return
    
    # Demo conversations similar to your C# code
    demo_conversations = [
        "What's the weather in Amsterdam?",
        "Is it likely to rain in London?",
        "Can you calculate 15 * 8 + 12?", 
        "What about the weather in Tokyo?",
        "Please compute (100 - 25) / 5"
    ]
    
    # Create a thread for the conversation
    print(f"\n🧵 Starting conversation with Main Agent: {main_agent.name}")
    print(f"   Agent ID: {main_agent.id}")
    print("-" * 60)
    
    current_thread = None
    
    for i, message in enumerate(demo_conversations, 1):
        print(f"\n💬 User ({i}): {message}")
        
        response, current_thread = await run_agent_conversation(
            main_agent, 
            message, 
            current_thread
        )
        
        print(f"🤖 {main_agent.name}: {response}")
        
        # Small delay for readability
        await asyncio.sleep(1)
    
    print(f"\n🎉 Demo completed!")
    print(f"📊 All conversations happened in thread: {current_thread}")
    print(f"🔍 Check AI Foundry portal for traces")
    
    return current_thread

# Run the agents demo
demo_thread = await demo_agents_with_tools()

In [ ]:
# Demo: Individual specialized agents
async def demo_individual_agents():
    """Demo individual weather and calculator agents"""
    print("🎯 Individual Agent Demonstrations")
    print("=" * 40)
    
    if not weather_agent or not calculator_agent:
        print("❌ Individual agents not available")
        return
    
    # Weather Agent Demo
    print(f"\n🌤️  Testing Weather Agent: {weather_agent.name}")
    print(f"   Agent ID: {weather_agent.id}")
    
    weather_query = "What's the weather in Sydney?"
    print(f"💬 User: {weather_query}")
    
    weather_response, weather_thread = await run_agent_conversation(
        weather_agent,
        weather_query
    )
    print(f"🌤️  Weather Agent: {weather_response}")
    
    # Calculator Agent Demo  
    print(f"\n🧮 Testing Calculator Agent: {calculator_agent.name}")
    print(f"   Agent ID: {calculator_agent.id}")
    
    calc_query = "What is 25 * 4 + 100?"
    print(f"💬 User: {calc_query}")
    
    calc_response, calc_thread = await run_agent_conversation(
        calculator_agent,
        calc_query
    )
    print(f"🧮 Calculator Agent: {calc_response}")
    
    print(f"\n📊 Summary:")
    print(f"   Weather conversation in thread: {weather_thread}")
    print(f"   Calculator conversation in thread: {calc_thread}")
    print(f"✅ Both specialized agents working correctly!")

# Demo individual agents
await demo_individual_agents()

In [ ]:
# Note: This is a legacy implementation - use the Azure AI Agents above instead
# This cell is kept for reference but shouldn't be executed

print("⚠️  This legacy AgentWithTools class is not used in the current implementation.")
print("✅ Use the Azure AI Agents created above instead (weather_agent, calculator_agent, main_agent)")

# Legacy class definition (commented out to avoid errors)
"""
class AgentWithTools:
    # This was the old custom implementation
    # Now we use real Azure AI Agents instead
    pass
"""

In [ ]:
# Note: Tool detection is handled by Azure AI Agents automatically
# This cell is kept for reference but is not needed for the current implementation

print("⚠️  Tool detection is handled automatically by Azure AI Agents.")
print("✅ The agents above will automatically detect and use tools based on user input.")

# Legacy tool detection (commented out to avoid errors)
"""
def detect_tool_usage(message: str) -> tuple[str, str]:
    # This was used for the old custom implementation
    # Azure AI Agents handle this automatically
    pass
"""

In [ ]:
# Note: This legacy method is not used in the current implementation
# The Azure AI Agents handle conversations automatically

print("⚠️  This legacy run_agent_with_thread method is not used.")
print("✅ Use the run_agent_conversation function above instead.")

# Legacy method (commented out to avoid errors)
"""
async def run_agent_with_thread(agent, message: str, thread_id: str) -> str:
    # This was used for the old custom implementation
    # Now we use run_agent_conversation with Azure AI Agents
    pass
"""

In [ ]:
# Demo: Quick test using Azure AI Agents
async def quick_test():
    """Quick test of Azure AI agents"""
    print("🧪 Quick Azure AI Agent Test")
    print("=" * 30)
    
    if not main_agent:
        print("❌ Main agent not available - check Azure AI Projects configuration")
        return None
    
    # Test message
    test_message = "What's the weather in Tokyo?"
    print(f"\n💬 User: {test_message}")
    
    # Get response using Azure AI agent
    response, thread_id = await run_agent_conversation(main_agent, test_message)
    print(f"🤖 {main_agent.name}: {response}")
    print(f"📝 Conversation in thread: {thread_id}")
    
    return thread_id

# Run quick test
test_thread = await quick_test()

In [ ]:
# Demo: Full conversation with Azure AI Agents
async def full_demo():
    """Demonstrate Azure AI agents with tools and threads"""
    print("🤖 Azure AI Agents with Tools and Threads - Full Demo")
    print("=" * 55)
    
    if not main_agent:
        print("❌ Main agent not available - check Azure AI Projects configuration")
        return None
    
    # Demo conversations
    demo_messages = [
        "What's the weather in Amsterdam?",
        "Is it likely to rain in London?", 
        "Can you calculate 15 * 8 + 12?",
        "What about the weather in Tokyo?",
        "Please compute (100 - 25) / 5",
        "How's the weather in Sydney?"
    ]
    
    print(f"\n🎯 Starting conversation with {main_agent.name}")
    print(f"   Agent ID: {main_agent.id}")
    print("-" * 40)
    
    current_thread = None
    
    for i, message in enumerate(demo_messages, 1):
        print(f"\n💬 User ({i}): {message}")
        
        response, current_thread = await run_agent_conversation(
            main_agent, 
            message, 
            current_thread
        )
        
        print(f"🤖 {main_agent.name}: {response}")
        
        # Small delay for better readability
        await asyncio.sleep(0.5)
    
    print(f"\n🎉 Demo completed!")
    print(f"📊 All conversations happened in thread: {current_thread}")
    
    return current_thread

# Run full demo
demo_thread = await full_demo()

In [ ]:
# Azure AI Agents thread analysis
async def analyze_agent_threads():
    """Analyze Azure AI agent conversations"""
    print("📊 Azure AI Agent Analysis")
    print("=" * 30)
    
    if not client:
        print("❌ Azure AI Projects client not available")
        return
    
    print("✅ Azure AI Agents are managed by the Azure AI Projects service")
    print("🔍 Thread management and conversation history are handled automatically")
    print("📈 View detailed analytics in the AI Foundry portal")
    
    # Show agent information
    agents = [
        ("Weather Agent", weather_agent),
        ("Calculator Agent", calculator_agent), 
        ("Main Agent", main_agent)
    ]
    
    for name, agent in agents:
        if agent:
            print(f"\n🤖 {name}:")
            print(f"   ID: {agent.id}")
            print(f"   Model: {agent.model}")
            print(f"   Tools: {len(agent.tools) if agent.tools else 0}")
        else:
            print(f"\n❌ {name}: Not available")

# Analyze Azure AI agents
await analyze_agent_threads()

In [ ]:
# Interactive chat with Azure AI Agents
async def interactive_chat():
    """Interactive chat with Azure AI agents"""
    print("🎮 Interactive Azure AI Agent Chat")
    print("=" * 35)
    
    if not main_agent:
        print("❌ Main agent not available - check Azure AI Projects configuration")
        return
    
    print("Commands:")
    print("  - Type your message to chat")
    print("  - 'weather' to switch to weather agent")
    print("  - 'calculator' to switch to calculator agent")
    print("  - 'main' to switch to main agent")
    print("  - 'quit' to exit")
    
    current_agent = main_agent
    current_thread = None
    
    while True:
        agent_name = current_agent.name if current_agent else "No Agent"
        user_input = input(f"\n💬 You ({agent_name}): ").strip()
        
        if user_input.lower() == 'quit':
            break
        elif user_input.lower() == 'weather' and weather_agent:
            current_agent = weather_agent
            current_thread = None  # Start new thread
            print(f"🌤️  Switched to {current_agent.name}")
            continue
        elif user_input.lower() == 'calculator' and calculator_agent:
            current_agent = calculator_agent  
            current_thread = None  # Start new thread
            print(f"🧮 Switched to {current_agent.name}")
            continue
        elif user_input.lower() == 'main' and main_agent:
            current_agent = main_agent
            current_thread = None  # Start new thread
            print(f"🤖 Switched to {current_agent.name}")
            continue
        elif user_input == '':
            continue
        
        if current_agent:
            response, current_thread = await run_agent_conversation(
                current_agent, 
                user_input, 
                current_thread
            )
            print(f"🤖 {current_agent.name}: {response}")
        else:
            print("❌ No agent available")
    
    print("👋 Chat ended!")

# Uncomment the line below to start interactive chat
# await interactive_chat()

In [ ]:
# Environment check and summary
def check_environment():
    """Check environment configuration for Azure AI agents"""
    print("🔍 Azure AI Agent Environment Check")
    print("=" * 35)
    
    # Check Azure AI Projects configuration
    required_vars = [
        "AZURE_AI_PROJECT_ENDPOINT",
        "AZURE_AI_MODEL_DEPLOYMENT_NAME"
    ]
    
    print("📋 Azure AI Projects Configuration:")
    for var in required_vars:
        value = os.getenv(var)
        if value:
            print(f"✅ {var}: {value[:50]}...")
        else:
            print(f"❌ {var}: Not set")
    
    # Check tracing
    print(f"\n📊 Tracing Status:")
    try:
        from opentelemetry import trace
        tracer_provider = trace.get_tracer_provider()
        print(f"✅ OpenTelemetry tracer provider: {type(tracer_provider).__name__}")
    except Exception as e:
        print(f"❌ Tracing not available: {e}")
    
    # Check agents
    print(f"\n🤖 Azure AI Agents Status:")
    agents = [
        ("Weather Agent", weather_agent),
        ("Calculator Agent", calculator_agent),
        ("Main Agent", main_agent)
    ]
    
    available_agents = 0
    for name, agent in agents:
        if agent:
            print(f"✅ {name}: Available (ID: {agent.id})")
            available_agents += 1
        else:
            print(f"❌ {name}: Not available")
    
    print(f"\n🎯 Summary:")
    print(f"   • Available agents: {available_agents}/3")
    print(f"   • Azure AI Projects client: {'✅ Connected' if client else '❌ Not available'}")
    print(f"   • Tracing configured: {'✅ Yes' if 'opentelemetry' in globals() else '❌ No'}")
    print(f"\n🚀 Ready for AI Foundry! Check the portal for agent traces.")

check_environment()